<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sklearn import tree
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [3]:
from IPython.display import SVG, display, HTML

In [1]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/toy_coref_1.csv

--2021-05-03 13:10:19--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/toy_coref_1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 173699 (170K) [text/plain]
Saving to: ‘toy_coref_1.csv’

toy_coref_1.csv     100%[===================>] 169.63K  --.-KB/s    in 0.008s  

2021-05-03 13:10:19 (21.0 MB/s) - ‘toy_coref_1.csv’ saved [173699/173699]



In [154]:
cor_df = pd.read_csv('toy_coref_1.csv', delimiter='\t')

In [155]:
cor_df

,TOKEN,POST_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_POS,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS.1,SENT_VECTOR,NER,COREFERENCE_CLUSTER,Unnamed: 17,Unnamed: 18
0,На,10.0,-,-,-,-,PREP,case,рынок,NOUN,inan,accs,masc,NOUN,0.2611904942896217,0,0,NaN,NaN
1,рынок,10.0,inan,accs,masc,-,NOUN,obl,вернулся,VERB,-,-,masc,VERB,0.2611904942896217,0,0,NaN,NaN
2,снова,10.0,-,-,-,-,ADVB,advmod,вернулся,VERB,-,-,masc,VERB,0.2611904942896217,0,0,NaN,NaN
3,вернулся,10.0,-,-,masc,-,VERB,ROOT,вернулся,VERB,-,-,masc,VERB,0.2611904942896217,0,0,NaN,NaN
4,алимемазин,10.0,inan,nomn,masc,-,NOUN,nsubj,вернулся,VERB,-,-,masc,VERB,0.2611904942896217,B-MED,алимемазин,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1793,производства,46.0,inan,gent,neut,-,NOUN,ROOT,производства,NOUN,inan,gent,neut,NOUN,0.145365668859865,0,0,NaN,NaN
1794,hypnotic72,47.0,-,-,-,-,-,nsubj,писал,VERB,-,-,masc,VERB,0.145365668859865,0,0,NaN,NaN
1795,писал,47.0,-,-,masc,-,VERB,ROOT,писал,VERB,-,-,masc,VERB,0.046053907523552574,0,0,NaN,NaN
1796,а,47.0,-,-,-,-,CONJ,punct,писал,VERB,-,-,masc,VERB,0.046053907523552574,0,0,NaN,NaN


In [156]:
cor_df.isnull().sum()

TOKEN                     1
POST_NUM                  1
ANIMACY                   1
CASE                      1
GENDER                    1
PERSON                    1
POS                       1
DEPENDENCY                1
HEAD                      1
HEAD_POS                  1
HEAD_ANIMACY              1
HEAD_CASE                 1
HEAD_GENDER               1
HEAD_POS.1                1
SENT_VECTOR               0
NER                       1
COREFERENCE_CLUSTER       4
Unnamed: 17            1788
Unnamed: 18            1797
dtype: int64

In [157]:
cor_df = cor_df.iloc[:-5,:]
cor_df_test = cor_df.iloc[-5:-1,:]
cor_df = cor_df.drop(['Unnamed: 17', 'Unnamed: 18'], axis=1)
cor_df_test  = cor_df_test .drop(['Unnamed: 17', 'Unnamed: 18'], axis=1)

In [159]:
cor_df_test = cor_df_test.reset_index()
cor_df_test = cor_df_test.drop(['index'], axis=1)

In [161]:
cor_df_test = cor_df_test.drop(['level_0'], axis=1)
cor_df_test

,TOKEN,POST_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_POS,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS.1,SENT_VECTOR,NER,COREFERENCE_CLUSTER
0,что,46.0,-,-,-,-,CONJ,fixed,Так,CONJ,-,-,-,CONJ,0.145365668859865,0,0
1,дак,46.0,-,-,-,-,CONJ,nsubj,будет,VERB,-,-,-,VERB,0.145365668859865,0,даклатасвир
2,скоро,46.0,-,-,-,-,ADVB,advmod,будет,VERB,-,-,-,VERB,0.145365668859865,0,0
3,будет,46.0,-,-,-,3per,VERB,ROOT,будет,VERB,-,-,-,VERB,0.145365668859865,0,0


In [162]:
cor_df

,TOKEN,POST_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_POS,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS.1,SENT_VECTOR,NER,COREFERENCE_CLUSTER
0,На,10.0,-,-,-,-,PREP,case,рынок,NOUN,inan,accs,masc,NOUN,0.2611904942896217,0,0
1,рынок,10.0,inan,accs,masc,-,NOUN,obl,вернулся,VERB,-,-,masc,VERB,0.2611904942896217,0,0
2,снова,10.0,-,-,-,-,ADVB,advmod,вернулся,VERB,-,-,masc,VERB,0.2611904942896217,0,0
3,вернулся,10.0,-,-,masc,-,VERB,ROOT,вернулся,VERB,-,-,masc,VERB,0.2611904942896217,0,0
4,алимемазин,10.0,inan,nomn,masc,-,NOUN,nsubj,вернулся,VERB,-,-,masc,VERB,0.2611904942896217,B-MED,алимемазин
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1788,что,46.0,-,-,-,-,CONJ,fixed,Так,CONJ,-,-,-,CONJ,0.145365668859865,0,0
1789,дак,46.0,-,-,-,-,CONJ,nsubj,будет,VERB,-,-,-,VERB,0.145365668859865,0,даклатасвир
1790,скоро,46.0,-,-,-,-,ADVB,advmod,будет,VERB,-,-,-,VERB,0.145365668859865,0,0
1791,будет,46.0,-,-,-,3per,VERB,ROOT,будет,VERB,-,-,-,VERB,0.145365668859865,0,0


In [187]:
X = cor_df.drop(['COREFERENCE_CLUSTER', 'NER', 'TOKEN'], axis=1)
X_test = cor_df_test.drop(['COREFERENCE_CLUSTER', 'NER', 'TOKEN'], axis=1)
labels_dict = {}
test_labels_dict = {}

for i in X.columns:
  X[i], lbl = pd.factorize(X[i])
  labels_dict[i] = lbl

for i in X_test.columns:
  X_test[i], lbl = pd.factorize(X_test[i])
  test_labels_dict[i] = lbl 

y = cor_df['COREFERENCE_CLUSTER']
y, y_label = pd.factorize(y)

y_test = cor_df_test['COREFERENCE_CLUSTER']
y_test, y_test_label = pd.factorize(y_test)

In [165]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [166]:
len(y)

1793

In [167]:
label[-1]

'омез'

In [168]:
uniques

Index(['0', 'алимемазин', 'терален', 'сонапакс ', 'велаксин', 'тералиджен',
       'азалептин', 'ремерон', 'золендроновая кислота-B',
       'золендроновая кислота-I', 'золендроновая кислота', 'сторониц',
       'самарий', 'изотопы', 'HIFU-терапия', 'абраксан', 'гемцитабин',
       'капецитабин', 'капицетабин', 'Инорекан', 'софосбувир', 'даклатасвир',
       'рибавирин', 'велпатасвир', 'ледипасвир', 'омез'],
      dtype='object')

In [169]:
X

,POST_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_POS,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS.1,SENT_VECTOR
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,0,1,1,1,1,1,1,0,1,0
2,0,0,0,0,0,2,2,1,1,1,1,0,1,0
3,0,0,0,1,0,3,3,1,1,1,1,0,1,0
4,0,1,2,1,0,1,4,1,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1788,39,0,0,0,0,9,12,498,2,1,1,3,2,147
1789,39,0,0,0,0,9,4,310,1,1,1,3,1,147
1790,39,0,0,0,0,2,2,310,1,1,1,3,1,147
1791,39,0,0,0,2,3,3,310,1,1,1,3,1,147


In [170]:
labels_dict['HEAD_POS']

Index(['NOUN', 'VERB', 'CONJ', 'PRTS', '-', 'PRCL', 'ADJF', 'ADVB', 'PRED',
       'INFN', 'NPRO', 'ADJS', 'COMP', 'INTJ', 'NUMR', 'PREP', 'GRND', 'PRTF'],
      dtype='object')

In [171]:
clf = tree.DecisionTreeClassifier()
clf.fit(X, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [172]:
clf_ent = tree.DecisionTreeClassifier(criterion='entropy')
clf_ent.fit(X, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [176]:
from graphviz import Source
from sklearn import tree

In [186]:
# tree.plot_tree(clf)

In [181]:
# graph = Source(tree.export_graphviz(clf, out_file=None, 
#                                     feature_names=list(X), class_names=uniques, filled=True))

# display(SVG(graph.pipe(format='svg')))

In [183]:
clf.predict(X_test)

array([0, 0, 0, 0])

In [188]:
y_test

array([0, 1, 0, 0])

In [184]:
X_test

,POST_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_POS,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS.1,SENT_VECTOR
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,1,1,0,0,0,1,0
2,0,0,0,0,0,1,2,1,1,0,0,0,1,0
3,0,0,0,0,1,2,3,1,1,0,0,0,1,0
